# Hallucination Detection

<img src="https://live.staticflickr.com/65535/54208132682_73767c3560_b.jpg" alt="Embedded Photo" width="500">

*Image generated using DALL-E model.*

## Introduction

Language models help us with daily tasks such as correcting texts, writing code, or answering questions. They are also increasingly used in fields like medicine and education.

However, how can we know if the answers generated by them are correct? Language models do not always have full knowledge of a given topic, yet they can formulate answers that sound plausible but are actually misleading. Such incorrect answers are called hallucinations.

## Task

In this task, you will be working on detecting hallucinations in answers to factual questions generated by large language models (LLM).
You will analyze a dataset that will help you evaluate whether the answers generated by the language model are actually correct, or contain hallucinations.

Each example in the dataset contains:

- **Question** e.g. "What is the main responsibility of the US Department of Defense?"
- **Answer** e.g. "The main responsibility is national defense.."
- **Tokens** associated with answer generation.
- **Four alternative answers** generated by the same model with higher temperature.
- **Four alternative tokens** generated by the same model with higher temperature.
- **Four alternative probabilities** generated by the same model with higher temperature.
- **Etykietę (`is_correct`)** indicating whether the main answer is correct according to a trusted source. 

Example:
```json
[
    {
        "question_id": 34,
        "question": "What is the name of the low-cost carrier that operates as a wholly owned subsidiary of Singapore Airlines?",
        "answer": "Scoot is the low-cost carrier that operates as a wholly owned subsidiary of Singapore Airlines.",
        "tokens": [" Sco", "ot", " is", ..., " Airlines", ".", "\n"],
        "supporting_answers": [
            "As a wholly owned subsidiary of Singapore Airlines, <answer> Scoot </answer> stands as a low-cost carrier that revolutionized air travel in the region.",
            "Scoot, a subsidiary of <answer> Singapore Airlines </answer> , is the low-cost carrier that operates under the same brand.",
            "<answer> Scoot </answer> is the low-cost carrier that operates as a wholly owned subsidiary of Singapore Airlines.",
            "Singapore Airlines operates a low-cost subsidiary named <answer> Scoot </answer> , offering affordable and efficient air travel options to passengers."
        ],
        "supporting_tokens": [
            [" As", " a", ..., ".", "<answer>"],
            [" Sco", "ot", ..., " brand", ".", "\n"],
            ["<answer>", " Sco", ..., ".", "\n"],
            [" Singapore", " Airlines", ..., ".", "\n"]
        ],
        "supporting_probabilities": [
            [0.0029233775567263365, 0.8621460795402527, ..., 0.018515007570385933],
            [0.42073577642440796, 0.9999748468399048, ..., 0.9166142344474792],
            [0.3258324861526489, 0.9969879984855652, ..., 0.921079695224762],
            [0.11142394691705704, 0.960810661315918, ..., 0.9557166695594788]
        ],
        "is_correct": true
    },
    .
    .
    .
]
```

### Data
Data available to you in this task:

* `train.json` - a dataset containing 2967 questions and answers.
* `valid.json` - 990 additional questions.


### Evaluation Criteria

ROC AUC (ang. *Receiver Operating Characteristic Area Under Curve*) is a measure of the quality of a binary classifier. It shows the model's ability to distinguish between two classes - here, hallucination (false) and correct answer (true). 

- **ROC (Receiver Operating Characteristic)**: A plot showing the dependency between *True Positive Rate* (sensitivity) and *False Positive Rate* (1-specificity) at different decision thresholds.
- **AUC (Area Under Curve)**: The area under the ROC curve, which takes values from 0 to 1:
  - **1.0**: Perfect classifier.
  - **0.5**: Random classifier (lack of ability to distinguish classes).

The higher the AUC value, the better the model performs in classification.

You can earn between 0 and 100 points for this task. The result will be linearly scaled based on the ROC AUC value:

- **ROC AUC ≤ 0.7**: 0 points.
- **ROC AUC ≥ 0.82**: 100 points.
- **Values between 0.7 and 0.82**: linearly scaled.

The formula for the result:  
$$
\text{Punkty} = 
\begin{cases} 
0 & \text{dla } \text{ROC AUC} \leq 0.7 \\
100 \times \frac{\text{ROC AUC} - 0.7}{0.82 - 0.7} & \text{dla } 0.7 < \text{ROC AUC} < 0.82 \\
100 & \text{dla } \text{ROC AUC} \geq 0.82
\end{cases}
$$


## Limitations
* Your solution will be tested on the Platform without internet access and without GPU.
* Evaluation of your final solution on the Platform cannot take longer than 5 minutes without GPU.
* List of allowed libraries: `xgboost`, `scikit-learn`, `numpy`, `pandas`, `matplotlib`.


## Submission Files
This notebook should be completed with your solution (see function `predict_hallucinations`).

## Evaluation
Remember that during evaluation, the flag `FINAL_EVALUATION_MODE` will be set to `True`.

You can earn between 0 and 100 points for this task. The number of points you earn will be calculated based on the ROC AUC value on a secret test set on the Platform, rounded to the nearest integer. If your solution does not meet the above criteria or does not run correctly, you will receive 0 points for the task.


# Kod Startowy
W tej sekcji inicjalizujemy środowisko poprzez zaimportowanie potrzebnych bibliotek i funkcji. Przygotowany kod ułatwi Tobie efektywne operowanie na danych i budowanie właściwego rozwiązania.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

FINAL_EVALUATION_MODE = False  # W czasie sprawdzania twojego rozwiązania, zmienimy tą wartość na True

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import shutil

def download_data(train=("1TGEDaxw4GKfSq0fpqSk0wRpUSc8GgZN0", "train.json"),
                  valid=("1qrr7bZk6Uct8DeC-V8Bc1qD5su56ryFd", "valid.json")):
    """Pobiera zbiór danych z Google Drive i zapisuje go w folderze 'data'."""
    import gdown
    
    # Utwórz lub zresetuj folder 'data'
    if not os.path.exists('data'):
        os.makedirs('data')
    else:
        shutil.rmtree('data')
        os.makedirs('data')

    GDRIVE_DATA = [train, valid]
    
    for file_id, file_name in GDRIVE_DATA:        
        # Pobierz plik z Google Drive i zapisz go w folderze 'data'
        url = f'https://drive.google.com/uc?id={file_id}'
        output = f'data/{file_name}'
        gdown.download(url, output, quiet=False)
        
        print(f"Downloaded: {file_name}")

# Pobierz dane tylko jeśli nie jesteś w trybie FINAL_EVALUATION_MODE
if not FINAL_EVALUATION_MODE:
    download_data()


## Ładowanie Danych
Za pomocą poniższego kodu dane zostaną wczytane i odpowiednio przygotowane.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

def load_data(folder='data'):
    # Wczytaj dane z plików
    train_path = os.path.join(folder, 'train.json')
    valid_path = os.path.join(folder, 'valid.json')
    
    with open(train_path, 'r') as f:
        train = json.load(f)
    with open(valid_path, 'r') as f:
        valid = json.load(f)

    return train, valid

train, valid = load_data("data")

print(json.dumps(train[0], indent=2))

print(f"\nWszystkie przykłady treningowe: {len(train)}")
print(f"Wszystkie przykłady walidacyjne: {len(valid)}")

## Kod z Kryterium Oceniającym

Kod, zbliżony do poniższego, będzie używany do oceny rozwiązania na zbiorze testowym.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

def compute_score(roc_auc: float) -> float:
    """
    Oblicza wynik punktowy na podstawie wartości ROC AUC.

    :param roc_auc: Wartość float w zakresie [0.0, 1.0]
    :return: Wynik punktowy zgodny z określoną funkcją
    """
    if roc_auc <= 0.7:
        return 0
    elif 0.7 < roc_auc < 0.82:
        return int(round(100 * (roc_auc - 0.7) / (0.82 - 0.7)))
    else:
        return 100


def evaluate_algorithm(dataset, algorithm, verbose=False):
    """
    Ewaluacja algorytmu wykrywania halucynacji na podanym zbiorze danych.

    Parametry
    ----------
    dataset : list
        Oznaczony zbiór danych, gdzie każdy element to słownik zawierający klucz 'is_correct'.
    algorithm : callable
        Funkcja, która przyjmuje pojedynczy przykład (słownik) i zwraca prawdopodobieństwo halucynacji.
    verbose : bool
        Jeśli True, wypisuje dodatkowe informacje dla każdego przykładu oraz podsumowanie.

    Zwraca
    -------
    roc_auc : float
        Wartość pola pod krzywą ROC (ROC AUC) dla predykcji.
    """
    predicted_ys = [] # Lista przechowująca przewidywane prawdopodobieństwa halucynacji

    for i, entry in enumerate(dataset):
        # Tworzenie kopii próbki i usunięcie etykiety, aby uzyskać dane wejściowe bez oznaczeń
        sample_unlabeled = dict(entry)
        sample_unlabeled.pop('is_correct', None)

        try:
            # Przewidywanie prawdopodobieństwa dla pojedynczej próbki
            pred_prob = algorithm(sample_unlabeled)
            predicted_ys.append(pred_prob)

        except Exception as e:
            # Jeśli wystąpi błąd, domyślnie ustawiamy prawdopodobieństwo na 0.5
            predicted_ys.append(0.5)
            if verbose:
                print(f"Sample {i} => Error: {e}")

    predicted_ys = np.array(predicted_ys, dtype=np.float32)
    ys = []
    for entry in dataset:
        ys.append(1 if entry.get('is_correct') else 0)
    ys = np.array(ys, dtype=np.int32)
    
    # Obliczenie metryki ROC AUC
    roc_auc = roc_auc_score(ys, predicted_ys)

    # Obliczenie końcowego wyniku na podstawie ROC AUC
    points = compute_score(roc_auc)

    if verbose:
        print(f"\nLiczba próbek: {len(dataset)}")
        print(f"ROC AUC: {roc_auc:.4f}")
        print(f"Wynik punktowy: {points}")

    return points

# Twoje Rozwiązanie
W tej sekcji należy umieścić Twoje rozwiązanie. Wprowadzaj zmiany wyłącznie tutaj!

In [ ]:
# TODO: Użyj danych treningowych, aby stworzyć tutaj model lub algorytm.

def predict_hallucinations(sample):
    # TODO: Uruchom swój model lub algorytm na tym zestawie danych.
    # TODO: Zwróć listę prawdopodobieństw dla każdego przykładu w zestawie danych.
    
    prediction = 0.5
    return prediction


# Ewaluacja

Uruchomienie poniższej komórki pozwoli sprawdzić, ile punktów zdobyłoby Twoje rozwiązanie na danych walidacyjnych. Przed wysłaniem upewnij się, że cały notebook wykonuje się od początku do końca bez błędów i bez konieczności ingerencji użytkownika po wybraniu opcji "Run All".

In [ ]:
if not FINAL_EVALUATION_MODE:
    roc_auc = evaluate_algorithm(valid, predict_hallucinations, verbose=True)

Podczas sprawdzania model zostanie zapisany jako `your_model.pkl` i oceniony na zbiorze testowym.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################
if FINAL_EVALUATION_MODE:      
    import cloudpickle
      
    OUTPUT_PATH = "file_output"
    FUNCTION_FILENAME = "your_model.pkl"
    FUNCTION_OUTPUT_PATH = os.path.join(OUTPUT_PATH, FUNCTION_FILENAME)

    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)

    with open(FUNCTION_OUTPUT_PATH, "wb") as f:
        cloudpickle.dump(predict_hallucinations, f)